# A Python software engineer

Some insights. This is the beta verions:
- Sometimes the code interpreter cannot generate output files, it depends on the prompting.
- Sometimes, the source code generated have sintax errors.

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
import os

# en windows es necesario especificar la ruta del .env
#load_dotenv(Path("C:\projects\rag\.env"))
load_dotenv(Path("/home/vicente/projects/agents_ai/.env"))
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

This is an example of a Python software engineer who can refactor, develop test units and run test units.

In [2]:
file = client.files.create(
  file=open("play_tetris.py", "rb"),
  purpose='assistants'
)

Some times, we cannot downlod the outputfile. We must ensure to include in instructinos, the name of the file and "Make the file id available to download".

In [3]:
output_file_name = "play_tetris_refactored.py"
assistant_instruction = r"""Generate {} file, always. Make the file id available to download. You are software engineer, your goal is to help another engineer write code using Python. 
  Focus on writting clean, performant and concise code to solve the problem.  """.format(output_file_name)

assistant = client.beta.assistants.create(
  name="Code writer assistant",
  description="Code writer assistant",  
  model="gpt-4o",
  instructions = assistant_instruction,
  tools=[{"type": "code_interpreter"}],
  tool_resources={
    "code_interpreter": {
      "file_ids": [file.id]
    }
  }  
)

In [4]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "Refactor this file.",
      "attachments": [
        {
          "file_id": file.id,
          "tools": [{"type": "code_interpreter"}]
        }
      ]
    }
  ]
)

In [5]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id
)

In [6]:
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_EdKxMvum4sKIVzDprolUd25p', assistant_id='asst_0cmrtBfRVkwWsRxiMivI8uCQ', attachments=[Attachment(file_id='file-1XhAEx2KrIgJyehPhfRQ7aUa', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FilePathAnnotation(end_index=191, file_path=FilePath(file_id='file-1XhAEx2KrIgJyehPhfRQ7aUa'), start_index=148, text='sandbox:/mnt/data/play_tetris_refactored.py', type='file_path')], value='The refactored code has been saved to `play_tetris_refactored.py`. You can download it using the link below:\n\n[Download the refactored Tetris game](sandbox:/mnt/data/play_tetris_refactored.py)'), type='text')], created_at=1725983895, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_Kl6h8HT84MS3LeI6yHr2g6Ae', status=None, thread_id='thread_cg7oYa81eOxoGP96m9DCkSig'), Message(id='msg_wBI5SnTjEIEjnU5fJXZy0YSl', assistant_id='asst_0cmr

Parsing results

In [7]:
output = messages.data[0].content[0].text.value
print(messages.data[0].content[0].text)
print(messages.data[0].content[0].text.annotations)
print(output)

Text(annotations=[FilePathAnnotation(end_index=191, file_path=FilePath(file_id='file-1XhAEx2KrIgJyehPhfRQ7aUa'), start_index=148, text='sandbox:/mnt/data/play_tetris_refactored.py', type='file_path')], value='The refactored code has been saved to `play_tetris_refactored.py`. You can download it using the link below:\n\n[Download the refactored Tetris game](sandbox:/mnt/data/play_tetris_refactored.py)')
[FilePathAnnotation(end_index=191, file_path=FilePath(file_id='file-1XhAEx2KrIgJyehPhfRQ7aUa'), start_index=148, text='sandbox:/mnt/data/play_tetris_refactored.py', type='file_path')]
The refactored code has been saved to `play_tetris_refactored.py`. You can download it using the link below:

[Download the refactored Tetris game](sandbox:/mnt/data/play_tetris_refactored.py)


In [8]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
file_path = messages.data[0].content[0].text.annotations[0].file_path.file_id
file_name = client.files.with_raw_response.retrieve_content(file_path)

with open(output_file_name, "wb") as file:  
    file.write(file_name.content) 

# Unit tests

We have two option, we can procced in a similar way as the beginning to get the unit test; however, in this case we can continue in the same thread.

In [9]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Write a simple unit test for that file. Generate a play_tetris_unit_test.py file, always. Make the file id available to download.")

In [10]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id
)

In [11]:
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_J0IVpZGZ3cLmsZzLXAyKHMhl', assistant_id='asst_0cmrtBfRVkwWsRxiMivI8uCQ', attachments=[Attachment(file_id='file-ZTVkkgnVjaEY4bMj7TGiID3K', tools=[CodeInterpreterTool(type='code_interpreter')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[FilePathAnnotation(end_index=191, file_path=FilePath(file_id='file-ZTVkkgnVjaEY4bMj7TGiID3K'), start_index=149, text='sandbox:/mnt/data/play_tetris_unit_test.py', type='file_path')], value='The unit test code has been saved to `play_tetris_unit_test.py`. You can download it using the link below:\n\n[Download the unit test for Tetris game](sandbox:/mnt/data/play_tetris_unit_test.py)'), type='text')], created_at=1725983942, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_TFQQ2w5uuNBQP8DRYXBg1DSA', status=None, thread_id='thread_cg7oYa81eOxoGP96m9DCkSig'), Message(id='msg_9zfXnpOHuBrncr1pGonKPAHX', assistant_id='asst_0cmrt

In [12]:
output = messages.data[0].content[0].text.value
print(messages.data[0].content[0].text)
print(messages.data[0].content[0].text.annotations)
print(output)

Text(annotations=[FilePathAnnotation(end_index=191, file_path=FilePath(file_id='file-ZTVkkgnVjaEY4bMj7TGiID3K'), start_index=149, text='sandbox:/mnt/data/play_tetris_unit_test.py', type='file_path')], value='The unit test code has been saved to `play_tetris_unit_test.py`. You can download it using the link below:\n\n[Download the unit test for Tetris game](sandbox:/mnt/data/play_tetris_unit_test.py)')
[FilePathAnnotation(end_index=191, file_path=FilePath(file_id='file-ZTVkkgnVjaEY4bMj7TGiID3K'), start_index=149, text='sandbox:/mnt/data/play_tetris_unit_test.py', type='file_path')]
The unit test code has been saved to `play_tetris_unit_test.py`. You can download it using the link below:

[Download the unit test for Tetris game](sandbox:/mnt/data/play_tetris_unit_test.py)


In [13]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
file_path = messages.data[0].content[0].text.annotations[0].file_path.file_id
file_name = client.files.with_raw_response.retrieve_content(file_path)

with open("play_tetris_unit_test.py", "wb") as file:  
    file.write(file_name.content) 

# Asking for running the unit test

I'm not sure if code interpreter is runing the unit test, because the resuts of the assistant are different when I run the unit test.

In [14]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Run the unit test and show the results")

In [15]:
run = client.beta.threads.runs.create_and_poll(
  thread_id=thread.id,
  assistant_id=assistant.id)

In [19]:
if run.status == 'completed': 
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
else:
  print(run.status)

failed


The results are different

In [17]:
output = messages.data[0].content[0].text.value
print(messages.data[0].content[0].text)
print(messages.data[0].content[0].text.annotations)
print(output)

Text(annotations=[FilePathAnnotation(end_index=191, file_path=FilePath(file_id='file-ZTVkkgnVjaEY4bMj7TGiID3K'), start_index=149, text='sandbox:/mnt/data/play_tetris_unit_test.py', type='file_path')], value='The unit test code has been saved to `play_tetris_unit_test.py`. You can download it using the link below:\n\n[Download the unit test for Tetris game](sandbox:/mnt/data/play_tetris_unit_test.py)')
[FilePathAnnotation(end_index=191, file_path=FilePath(file_id='file-ZTVkkgnVjaEY4bMj7TGiID3K'), start_index=149, text='sandbox:/mnt/data/play_tetris_unit_test.py', type='file_path')]
The unit test code has been saved to `play_tetris_unit_test.py`. You can download it using the link below:

[Download the unit test for Tetris game](sandbox:/mnt/data/play_tetris_unit_test.py)
